In [146]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasRegressor

In [202]:
df1516 = pd.read_csv('201516stats.csv')
df1617 = pd.read_csv('201617stats.csv')
df1718 = pd.read_csv('201718stats.csv')
df1819 = pd.read_csv('201819stats.csv')
df1920 = pd.read_csv('201920stats.csv')
df = pd.concat([df1516,df1617,df1718,df1819,df1920]).reset_index().drop(['fullname'], axis=1)
df['faceOffPercentage'].fillna(0, inplace=True)
df['shootingPercentage'].fillna(0, inplace=True)
df.dropna(inplace=True)
test_X = df[df['lastName'] == 'Crosby']
X = df.drop(['player_id', 'firstName', 'lastName', 'season', 'salary'], axis=1)
y = df.filter(['salary'])

new_X = test_X.drop(['player_id', 'firstName', 'lastName', 'season', 'salary', 'primaryPosition', 'shootsCatches'], axis=1)

In [139]:
onehot = OneHotEncoder()
X_pos = onehot.fit_transform(X[['primaryPosition']])
X_hand = onehot.fit_transform(X[['shootsCatches']])
df1 = pd.DataFrame.sparse.from_spmatrix(X_pos)
df2 = pd.DataFrame.sparse.from_spmatrix(X_hand)
df1.rename(columns={0: 'isC', 1: 'isD', 2: 'isLW', 3: 'isRW'}, inplace=True)
df2.rename(columns={0: 'shootsL', 1: 'shootsR'}, inplace=True)
df3 = pd.concat([df1, df2], axis=1)
X_new = pd.concat([df3.reset_index(), X.drop(['primaryPosition', 'shootsCatches'], axis=1).reset_index()], axis=1)
X_normalized=(X_new-X_new.min())/(X_new.max()-X_new.min())

In [171]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.30, random_state=5)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.50, random_state=5)

In [ ]:
clf = Lasso()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
MSE = mean_squared_error(y_test, y_pred)
results = y_test.copy()
results['pred'] = y_pred.tolist()
sns.scatterplot(x='salary', y='pred', data=results)

In [182]:
def get_model():
    model = Sequential()
    model.add(Dense(33, input_dim= 33, activation='relu'))
    model.add(Dense(22, kernel_initializer='normal'))
    model.add(Dense(11, kernel_initializer='normal'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    return model

model = get_model()
model.fit(X_train, y_train, epochs = 200, batch_size = 5)
y_pred = model.predict(X_val)

Epoch 1/200
597/597 [==============================] - 1s 772us/step - loss: 10597468995584.0000 - mean_absolute_error: 2394485.5000
Epoch 2/200
597/597 [==============================] - 1s 894us/step - loss: 7568793010176.0000 - mean_absolute_error: 1824915.2500
Epoch 3/200
597/597 [==============================] - 0s 808us/step - loss: 3876709466112.0000 - mean_absolute_error: 1505164.0000
Epoch 4/200
597/597 [==============================] - 0s 730us/step - loss: 3446218424320.0000 - mean_absolute_error: 1512911.1250
Epoch 5/200
597/597 [==============================] - 0s 776us/step - loss: 3301860179968.0000 - mean_absolute_error: 1469979.5000
Epoch 6/200
597/597 [==============================] - 0s 712us/step - loss: 3182119354368.0000 - mean_absolute_error: 1434391.0000
Epoch 7/200
597/597 [==============================] - 0s 698us/step - loss: 3074069102592.0000 - mean_absolute_error: 1391754.1250
Epoch 8/200
597/597 [==============================] - 0s 705us/step - loss

In [206]:
new_X['isC'] = 1
new_X['isD'] = 0
new_X['isLW'] = 0
new_X['isRW'] = 0
new_X['shootsL'] = 1
new_X['shootsR'] = 0
norm_X=(new_X-new_X.min())/(new_X.max()-new_X.min())
test_test = model.predict(norm_X)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_7/dense_27/Relu' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1264.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1264.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Jacky\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Jacky\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\traitlets\config\application.py", line 965, in launch_instance
      app.start()
    File "C:\Users\Jacky\AppData\Local